In [35]:
import os
from xml.etree import ElementTree as ET
from lxml import etree
import re
from loguru import logger
from zhon.hanzi import punctuation
import string
import cn2an

import json
from xml.sax.saxutils import unescape
import jieba
import jieba.analyse as analyse

import time
from gensim.models import KeyedVectors
import numpy as np

In [10]:
t0 = time.time()
word2vec = KeyedVectors.load('/root/pretrain-models/word2vec/Tencent_AILab_ChineseEmbedding.bin', mmap='r')
print(time.time() - t0)

6.660168409347534


In [14]:
title_exclude_list = [
    "交接",
    "串联",
    "新串联",
    "稍后",
    "提要",
    "开始语",
    "结束语",
    "（连放",
    "【连放"
]


def check_file(file):
    if os.path.exists(file):
        os.remove(file)
    fp = open(file, "w", encoding="utf-8")
    fp.close()


def parse_html(txt):
    # 解析txt
    xml_str = unescape(txt)  # html -> html str
    html = etree.HTML(xml_str)
    result = html.xpath("//p//text()")

    # 去掉[正文]末尾的空格
    result = [res.strip() for res in result]

    # 提取正文和导语之后的内容
    text = result[result.index("[正文]"):result.index("[导语]")]
    introduction = result[result.index("[导语]"):result.index("[编后]")]
    mark_txt = ""
    for introduction_str in introduction:
        if len(introduction_str) > 10:
            mark_txt = introduction_str
            break
    if mark_txt == "":
        for text_str in text:
            if len(text_str) > 10 and text_str[0] not in ("【", "[", "") and not text_str.startswith("素材"):
                mark_txt = text_str
                break
    last_str = ""
    for text_str in text[::-1]:
        if len(text_str) > 10 and text_str[0] not in ("【", "[", "") and not text_str.startswith("东方卫视"):
            last_str += text_str
#             logger.info(last_str)
            break
    if last_str == "":
        for introduction_str in introduction[::-1]:
            if len(introduction_str) > 10:
                last_str = introduction_str
                break
    txt_str = ""
    for text_enum in text:
           txt_str = txt_str + text_enum
    intro_str = ""
    for intro_enum in introduction:
           intro_str = intro_str + intro_enum

    for i in punctuation:
        txt_str = txt_str.replace(i, "")
    for i in string.punctuation:
        txt_str = txt_str.replace(i, "")
    txt_str = txt_str.replace(" ", "")
    txt_str = cn2an.transform(txt_str, "an2cn")

    for i in punctuation:
        intro_str = intro_str.replace(i, "")
    for i in string.punctuation:
        intro_str = intro_str.replace(i, "")
    intro_str = intro_str.replace(" ", "")
    intro_str = cn2an.transform(intro_str, "an2cn")
    return txt_str, intro_str, mark_txt, last_str


def analysis_series_bill(xml_path):
    # 解析xml
    root = ET.parse(xml_path).getroot()
    # 遍历txt
    data = []
    for node in root.iter("text"):
        # num
        num = node.find("num").text
        title = node.find("title").text
        txt = node.find("txt").text
        words = node.find("words").text

        # 解析txt
        text, introduction, mark_txt, last_txt = parse_html(txt)

        audiotime = node.find("audiotime").text
        runtime = node.find("runtime").text
        totaltime = node.find("totaltime").text

        # for exclude_title in title_exclude_list:
        #     if re.match(exclude_title, title, flags=0):
        #         mark_txt = ""

        line = {"num": num, "title": title, "txt": text, "introduction": introduction, "audio_time": audiotime,
                "runtime": runtime, "total_time": totaltime, "mark_txt": mark_txt, "last_txt": last_txt, "words": words}
        data.append(line)

    return data

In [31]:
xml_20 = analysis_series_bill('../data/news/20210620.xml')
xml_20

[{'num': '1',
  'title': '东方新闻提要',
  'txt': '正文',
  'introduction': '导语提要全国新冠疫苗接种累计已超过十亿剂次接种剂次和覆盖人群数量均居全球首位我国页岩油勘探获得重大突破中石油在鄂尔多斯盆地探明国内首个超十亿吨级大油田上海苏州河上二十二座桥梁景观提升改造工程即将收官以微更新打造有温度的滨水空间第二十四届上海国际电影节金爵奖揭晓中国电影东北虎荣获最佳影片云南北迁象群继续在原地小范围活动享受池塘日光浴专家初步判断象群正踏上返程路伊朗大选落幕强硬派莱希毫无悬念胜出如何摆脱美国制裁将是新总统上任后首要任务',
  'audio_time': '50',
  'runtime': '30',
  'total_time': '80',
  'mark_txt': '全国新冠疫苗接种 累计已超过10亿剂次，接种剂次\\和覆盖人群数量 均居全球首位。##',
  'last_txt': '伊朗大选落幕，"强硬派"\\莱希\\毫无悬念胜出，如何摆脱美国制裁，将是新总统上任后\\首要任务。##',
  'words': '275'},
 {'num': '2',
  'title': '【已锁定】摇臂过场音乐',
  'txt': '正文',
  'introduction': '导语',
  'audio_time': '0',
  'runtime': '10',
  'total_time': '10',
  'mark_txt': '',
  'last_txt': '',
  'words': '0'},
 {'num': '3',
  'title': '开始语（单人）',
  'txt': '正文',
  'introduction': '导语今天是二零二一年六月二十日星期日欢迎收看东方新闻我是小雪在上海的演播室向各位问好柏我是柏栩栩我们在上海的演播室向各位问好在快速浏览了今天的主要新闻之后我们来看详细报道',
  'audio_time': '13',
  'runtime': '0',
  'total_time': '13',
  'mark_txt': '今天是2021年 6月 20日 星期日，欢迎收看《东方新闻》，我是小雪，在上海的演播室向各位问好。',
  'last_txt': '在快速

In [32]:
def keyword_extract(text, remove_stopwords=True) -> dict:
    res = {
        'tf-idf': None,
        'textrank': None
    }
    res['tf-idf'] = analyse.extract_tags(text, topK=5)
    res['textrank'] = analyse.textrank(text, topK=5)
    return res

In [29]:
keyword_extract('导语提要全国新冠疫苗接种累计已超过十亿剂次接种剂次和覆盖人群数量均居全球首位我国页岩油勘探获得重大突破中石油在鄂尔多斯盆地探明国内首个超十亿吨级大油田上海苏州河上二十二座桥梁景观提升改造工程即将收官以微更新打造有温度的滨水空间第二十四届上海国际电影节金爵奖揭晓中国电影东北虎荣获最佳影片云南北迁象群继续在原地小范围活动享受池塘日光浴专家初步判断象群正踏上返程路伊朗大选落幕强硬派莱希毫无悬念胜出如何摆脱美国制裁将是新总统上任后首要任务')

{'tf-idf': ['剂次', '象群', '接种', '莱希', '强硬派'],
 'textrank': ['接种', '石油', '上海', '享受', '疫苗']}

In [ ]:
for idx in range(len(xml_20)-1):
    current_keyword = keyword_extract(xml_20[idx]['introduction'])
    next_keyword = keyword_extract(xml_20[idx + 1]['introduction'])
    current_keyword_vec = 

In [36]:
def keywords2vec(lst):
    s

array([ 0.20538543, -0.04878445, -0.08076064,  0.00641524,  0.04364172,
        0.08515599,  0.07804526,  0.05523099,  0.02419453, -0.06994568,
        0.09646476, -0.08158586, -0.02176837, -0.08105281,  0.05050642,
       -0.0333513 , -0.02549042, -0.05258693,  0.01552724, -0.08721827,
       -0.01479234, -0.01058973,  0.00477845,  0.0105822 , -0.04916339,
        0.07225498, -0.14384082, -0.05570898,  0.03700699,  0.05138197,
        0.0658015 , -0.05201208,  0.18906228,  0.02498668,  0.00171939,
       -0.02541356,  0.04652765, -0.02697231,  0.0334864 ,  0.03608272,
        0.06212977,  0.18081196,  0.12054916,  0.07518706, -0.08888339,
       -0.0582647 , -0.05751592, -0.12165747,  0.07903549,  0.07270325,
        0.02392989, -0.03164458, -0.0176028 ,  0.09749106,  0.08936533,
       -0.1300607 , -0.11932459,  0.02047466, -0.00388666,  0.02974929,
       -0.01217819,  0.10663098, -0.03610669, -0.03573864, -0.15395771,
       -0.10878102,  0.09194303, -0.04945636,  0.09857122, -0.04

In [68]:
def sim_cosin(v_o, v_t):
    return 0.5 + 0.5 * np.dot(v_o, v_t) / (np.linalg.norm(v_o) * np.linalg.norm(v_t)) if (np.linalg.norm(v_o) * np.linalg.norm(v_t)) != 0 else 0

In [70]:
sim_cosin(np.array(word2vec['接种']), np.array(word2vec['疫苗']))

0.9416744043438916

1.7320508075688772